In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
from scipy import stats
# %matplotlib inline
pd.options.display.max_columns = None
def increase_figsize(N, plt):
    params = plt.gcf()
    axes = params.get_axes()
    plSize = params.get_size_inches()
    params.set_size_inches( (plSize[0]*N, plSize[1]*N) )
def increase_figsize2(N, fig):
    params = fig
    axes = params.get_axes()
    plSize = params.get_size_inches()
    params.set_size_inches( (plSize[0]*N, plSize[1]*N) )
def increase_size_by(N, fontsize, plt):
    params = plt.gcf()
    axes = params.get_axes()
    plSize = params.get_size_inches()
    params.set_size_inches( (plSize[0]*N, plSize[1]*N) )
    for ax in axes:
        for item in ([ax.title, ax.xaxis.label, ax.yaxis.label] +
                ax.get_xticklabels() + ax.get_yticklabels()):
            item.set_fontsize(fontsize)
cbf = ['#377eb8', '#ff7f00', '#4daf4a',
                  '#f781bf', '#a65628', '#984ea3',
'#999999', '#e41a1c', '#dede00']
cbf_map = mpl.colors.ListedColormap(cbf)
rc = {'axes.facecolor': 'white', 'axes.edgecolor': '.15', 'axes.axisbelow': True, 'figure.facecolor': 'white',
      'axes.labelcolor': '.15',  'text.color': '.25',
      'patch.force_edgecolor': True, 'patch.edgecolor': 'w',
      'xtick.bottom': True, 'ytick.left': True, 'xtick.color': '.15', 'ytick.color': '.15',
      'grid.linestyle': '-', 'axes.grid': False, 'grid.color': '.8', 
      'font.size': 30, 
      'xtick.labelsize': 15, 'ytick.labelsize': 15, 
      'axes.titlesize': 22,
      'axes.labelsize': 15,
      'legend.fontsize': 18, 'legend.title_fontsize': 30,
      }
sns.set(rc=rc)
sns.set_palette(cbf)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
data_folder = '/home/hielke/bep/res/'
fig_folder = '/home/hielke/bep/figs/SECOND/'
first_order = 'C'
second_order = 0
save = False
title_mod = ""

In [4]:
df = pd.read_csv(data_folder + 'gene.matched.tsv.smallarraycheck', sep='\t', 
                 names=['genome_key', 'genome_id', 'contig_hit', 'c_1', 'c_2', 'spacer_id', 'hit_ident', 'spacer_size', 
                        'contig_size', '3_prime', '5_prime', 'array_id', 'array_size', 'array_confid', 'repeat_size', 'array_type', 'genome_type',
                        'PAM_confid', 'poss_PAM', 'PAM_side', 'phage_class', 'with_phage',  'gene_hit', 'gene_id', 'gene_class'],
                index_col=False, keep_default_na=False, dtype={'genome_id': str}).set_index('genome_id')
df.drop(['3_prime', '5_prime', 'genome_key',], axis=1, inplace=True)

In [12]:
df80 = df.query('hit_ident > 80 and gene_id != "NULL"')
dfgenehits = df80.groupby([df80.index, 'gene_id']).nth(0)
import csv
with open(data_folder + 'selection_for_gene_classification', 'w') as f:
    f.writelines(line + '\n' for line in set(df80.index))

In [13]:
# PERFORM SELECTION AS FOLLOWS:
# alias mtawk="mawk -F '\t' -v OFS='\t'"
# mtawk 'FNR==NR{hits[$0]=1;next;} $1 in hits' selection_for_gene_classification gene_classification > gene_classification.sel
dfgenes = pd.read_csv(data_folder + 'gene_classification.sel', sep='\t', names=['gene_id', 's', 'e', 'side', 'gene_class'])

In [14]:
gene_classes = dfgenes['gene_class'].value_counts()

In [94]:
# GENE ENRICHEMENT
# UNIQUE GENES
# CHI SQUARE TEST
gene_class_hits = dfgenehits['gene_class'].values_counts()
# gene_class_hits.sort_values(ascending=False
from scipy.stats.mstats import chisquare



gene_class
UNCLASSIFIED                                           5442
UNKNOWN                                                3227
P-loop containing nucleoside triphosphate hydrolase     812
Bacteriophage Lambda NinG protein                       498
DNA breaking-rejoining enzyme, catalytic core           419
Name: gene_class, dtype: int64

In [143]:
test = gene_class_hits.reindex(index=gene_classes.index, fill_value=0)


16811